<a href="https://colab.research.google.com/github/tabba98/neural-network/blob/main/VoxNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 18.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchmetrics.classification import Accuracy
from torchmetrics import ConfusionMatrix



#for plotting
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import confusion_matrix

import scipy
from scipy.ndimage import rotate

import time

In [3]:
%%capture
!wget https://www.dropbox.com/s/ja56cvf3x4mkf1t/modelnet10_voxelized_32.npz

In [4]:
class VoxelDataset(Dataset):
    def __init__(self, train = True):
      if train:
          tmp = np.load("modelnet10_voxelized_32.npz")
          self.data = tmp["X_train"]
          self.label = tmp["Y_train"]
          del tmp
      else:
          tmp = np.load("modelnet10_voxelized_32.npz")
          self.data = tmp["X_test"]
          self.label = tmp["Y_test"]
          del tmp
        
                
    def __len__(self):
        return len(self.label)

    def __preproc__(self, voxels):
        
        #flip x
        if np.random.randint(2):
            voxels = np.flip(voxels, axis=0)
        
        #flip y
        if np.random.randint(2):
            voxels = np.flip(voxels, axis=1)
        
        angle = 360 * np.random.random_sample(1)[0]
        
        voxels = rotate(voxels, axes=(0, 1), angle=angle, cval=0.0, reshape=False)
        
        
        return voxels.copy()

    def __getitem__(self, idx):
        label = self.label[idx]
        voxels = self.data[idx]
        voxels = self.__preproc__(voxels)
        voxels = np.expand_dims(voxels, axis=0)
        voxels = torch.tensor(voxels).float()
        return voxels, label

In [16]:
from collections import OrderedDict
class VoxNet(nn.Module):
    def __init__(self):
        super(VoxNet, self).__init__()
        n_classes = 10
        input_shape = (32,32,32)
        self.feat = torch.nn.Sequential(OrderedDict([
            ('conv3d_1', torch.nn.Conv3d(in_channels=1, out_channels=32, kernel_size=5, stride=2)),
            ('relu1', torch.nn.ReLU()),
            ('drop1', torch.nn.Dropout(p=0.2)),
            ('conv3d_2', torch.nn.Conv3d(in_channels=32, out_channels=32, kernel_size=3)),
            ('relu2', torch.nn.ReLU()),
            ('pool2', torch.nn.MaxPool3d(2)),
            ('drop2', torch.nn.Dropout(p=0.3))
        ]))
        
        x = self.feat(torch.autograd.Variable(torch.rand((1, 1) + input_shape)))
        dim_feat = 1
        for n in x.size()[1:]:
            dim_feat *= n

        self.mlp = torch.nn.Sequential(OrderedDict([
            ('fc1', torch.nn.Linear(dim_feat, 128)),
            ('relu1', torch.nn.ReLU()),
            ('drop3', torch.nn.Dropout(p=0.4)),
            ('fc2', torch.nn.Linear(128, n_classes))
        ]))

    def forward(self, x):
        x = self.feat(x)
        x = x.view(x.size(0), -1)
        x = self.mlp(x)
        return x


In [37]:
from torch.optim import SGD, Adam
from torch.nn import BCEWithLogitsLoss
from tqdm import tqdm

def blue(x): return '\033[94m' + x + '\033[0m'

train_ds = VoxelDataset(train=True)
test_ds = VoxelDataset(train=False)
train_dataloader = DataLoader(dataset=train_ds, batch_size=32, shuffle=True, drop_last=True)
test_dataloader = DataLoader(dataset=test_ds, batch_size=32)

model = VoxNet()
opt = SGD(model.parameters(), lr=1e-2, weight_decay = 0)
loss_fn = F.cross_entropy
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
epochs=10
best_val = np.inf

num_batch = len(train_ds) / 32
print(num_batch)

for epoch in range(epochs):
    temp_correct = 0
    temp_testset = 0
    temp_correct_test = 0
    temp_testset_test = 0
    for i, data in tqdm(enumerate(train_dataloader, 0)):
        inputs, labels = data[0], data[1]
        inputs = inputs.to(device)
        labels = labels.to(device)

        # 梯度清零
        opt.zero_grad()

        # 网络切换训练模型
        voxnet = model.train()
        pred = voxnet(inputs)  # torch.Size([256, 10])

        # 计算损失函数

        loss = F.cross_entropy(pred,labels)

        # 反向传播, 更新权重
        loss.backward()
        opt.step()

        # 计算该batch的预测准确率
        pred_choice = pred.data.max(1)[1]
        correct = pred_choice.eq(labels.data).cpu().sum()
        #print('[%d: %d/%d] train loss: %f accuracy: %f' % (epoch, i+1, num_batch, loss.item(), correct.item() / 32))
        temp_correct += correct.item()
        temp_testset += inputs.size()[0]
        
    
    for j, sample in tqdm(enumerate(test_dataloader, 0)):    
        inputs_test, labels_test = sample[0], sample[1]
        inputs_test = inputs_test.to(device)
        labels_test = labels_test.to(device)
        inputs_test = inputs_test.float()  # 转float, torch.Size([256, 1, 32, 32, 32])
        voxnet = voxnet.eval()
        pred_test = voxnet(inputs_test)
        loss_test = F.nll_loss(pred_test, labels_test)
        pred_choice_test = pred_test.data.max(1)[1]
        correct_test = pred_choice_test.eq(labels_test.data).cpu().sum()
        
        temp_correct_test += correct_test.item()
        temp_testset_test += inputs_test.size()[0]
    
    print("epoch %d: train accuracy %f" % (epoch, temp_correct / float(temp_testset)))
    print("epoch %d: test accuracy %f" % (epoch, temp_correct_test / float(temp_testset_test)))

    


total_correct = 0
total_testset = 0

for i, data in tqdm(enumerate(test_dataloader, 0)):
    inputs, labels = data[0], data[1]
    inputs = inputs.to(device)
    labels = labels.to(device)
    inputs = inputs.float()  # 转float, torch.Size([256, 1, 32, 32, 32])

    voxnet = voxnet.eval()
    pred = voxnet(inputs)
    pred_choice = pred.data.max(1)[1]
    correct = pred_choice.eq(labels.data).cpu().sum()
    total_correct += correct.item()
    total_testset += inputs.size()[0]

print("final accuracy {}".format(total_correct / float(total_testset)))



124.71875


124it [01:08,  1.81it/s]
29it [00:08,  3.49it/s]


epoch 0 train accuracy 0.355595
epoch 0 test accuracy 0.549559


124it [01:11,  1.72it/s]
29it [00:08,  3.53it/s]


epoch 1 train accuracy 0.623740
epoch 1 test accuracy 0.674009


...

epoch 8 train accuracy 0.811240
epoch 8 test accuracy 0.805066


124it [01:10,  1.77it/s]
29it [00:08,  3.45it/s]


epoch 9 train accuracy 0.824345
epoch 9 test accuracy 0.818282


29it [00:08,  3.45it/s]

final accuracy 0.8105726872246696
